In [ ]:
#r "nuget:Microsoft.Spark"
#r "nuget:Microsoft.ML"

In [ ]:
using System;
using System.Collections.Generic;
using System.Linq;
using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.Spark;
using Microsoft.Spark.Sql;
using Microsoft.Spark.Sql.Types;
using static Microsoft.Spark.Sql.Functions;

// run spark : spark-submit --class org.apache.spark.deploy.dotnet.DotnetRunner --master local D:\3bStudio\Sandbox\spark-program\FirstSparkProgram\bin\Debug\net6.0\microsoft-spark-3-0_2.12-2.1.0.jar debug
// see Spark portal at : http://localhost:4040
var spark = SparkSession
    .Builder()
    .AppName("spark-use-ml-model")
    .GetOrCreate();

spark.SparkContext.SetLogLevel("WARN");

In [ ]:
var inputSchema = new StructType(new[]
    {
        new StructField("age", new FloatType()),
        new StructField("cholesterol", new FloatType()),
        new StructField("restingBP", new FloatType()),
        new StructField("fastingBS", new FloatType())
    });

DataFrame df = spark.Read().Schema(inputSchema).Json("D:/3bStudio/Sandbox/3bs-spark-training/resources/ml-input.json");

var schema = df.Schema();
Console.WriteLine(schema.SimpleString);

IEnumerable<Row> rows = df.Collect();
foreach (Row row in rows)
{
    Console.WriteLine(row);
}

struct<age:float,cholesterol:float,restingBP:float,fastingBS:float>
[40,289,140,0]


In [ ]:
MLContext mlContext = new MLContext();
ITransformer model = mlContext.Model.Load("D:/2.Sandbox/3bs-spark-training/resources/heart-classification.zip", out DataViewSchema schema);
_predictionEngine = mlContext.Model.CreatePredictionEngine<ModelInput, ModelOutput>(model);

static bool predict(string text)
{
    ModelInput input = new ModelInput
    {
        ReviewText = text
    };

    return _predictionEngine.Predict(input).Prediction;
}

In [ ]:
spark.Udf()
    .Register<string, bool>("MLudf", predict);

// Use Spark SQL to call ML.NET UDF
df.CreateOrReplaceTempView("Heart-data");
DataFrame sqlDf = spark.Sql("SELECT ReviewText, MLudf(ReviewText) FROM Heart-data");
sqlDf.Show();

// Print out first 20 rows of data
// Prevent data getting cut off by setting truncate = 0
sqlDf.Show(20, 0, false);

spark.Stop();